# Build LLM Agent combining Reasoning and Action (ReAct) framework using LangChain

ReAct is technique which enable LLMs to do reasoning and take task specific actions. It combines chain of thought reasoning with action planning.

It enables LLMs to generate reasoning traces and task-specific actions, leveraging the synergy between them. This approach demonstrates superior performance over baselines in various tasks, overcoming issues like hallucination and error propagation.

In the notebook we will use OPENAI ChatGPT model.

https://research.google/blog/react-synergizing-reasoning-and-acting-in-language-models/


In [6]:
#!pip install openai langchain wikipedia
#!pip install langchain-community

In [18]:
import os
OPENAI_API_KEY = "xxx"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or getpass("OpenAI API key: ")

In [19]:
import openai
import os
openai.api_key  = os.getenv('OPENAI_API_KEY')

Langchain has the concept of Agent. The core idea of agent is to use a language model to choose a sequence of actions to take.

A language model agent is used as a reasoning engine to determine which actions to take and in which order. Langchain has an inbuilt agent called DocStore Explorer whose functionality aligns with the original ReAct research paper, for the Wikipedia use case.

The DocStoreExplorer agent interacts with a document storage system (like Wikipedia), using two specific tools , a Search tool and a Lookup tool. The Search tool is responsible for locating a document, whereas the Lookup tool retrieves a term from the most recently discovered document.

Initialize this doc store with Wikipedia as a document store.

In [20]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
docstore = DocstoreExplorer(Wikipedia())

Another concept that Langchain provides is called tools. Tools are interfaces that an agent can use to interact with the outer world. We can create our own tool for Wikipedia API. In the tool specification, we can specify which function agent to call for interaction with outer world.

In [21]:
from langchain.agents import Tool
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

Instantiate a GPT-4 model as a LLM for our application.

In [22]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4")

Initialize the agent with tools, the LLM and agent type.

In [23]:
from langchain.agents import AgentType, initialize_agent
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

When we pass our question to our ReAct agent, the Agent will interact with the OPENAI model using the API.

It will also pass tools information (description of functions) to the model as additional argument in a prompt. In the case of OPENAI it is called OPENAI functions.

If the model wants to search or lookup actions, then it returns that function description with function arguments (if any) to the agent.

Then the agent calls that function using the tool and calls the  Model again with that information. It will do this in iterative manner as explained above.

All this magic is done by agent for us.

In [24]:
question = "What profession does Elon Musk and Mark Zuckerburg have in common?"
result = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Elon Musk and Mark Zuckerberg, find their professions, then find the profession they have in common.
Action: Search[Elon Musk]
Observation: Could not find [Elon Musk]. Similar: ['Elon Musk', 'Wealth of Elon Musk', 'Elon Musk salute controversy', 'Protests against Elon Musk', 'Views of Elon Musk', 'Twitter under Elon Musk', 'Errol Musk', 'Acquisition of Twitter by Elon Musk', 'Business career of Elon Musk', 'Elon Musk (Isaacson book)']
Thought:I misspelled Elon Musk. I will correct it and search again.
Action: Search[Elon Musk]
Observation: Could not find [Elon Musk]. Similar: ['Elon Musk', 'Wealth of Elon Musk', 'Elon Musk salute controversy', 'Protests against Elon Musk', 'Views of Elon Musk', 'Twitter under Elon Musk', 'Errol Musk', 'Acquisition of Twitter by Elon Musk', 'Business career of Elon Musk', 'Elon Musk (Isaacson book)']
Thought:The search results seem to be repeating. I will select 'Business career of Elon M

#### We can finally get the answer

In [25]:
result

'businessman and CEO'